In [5]:
import pandas as pd
from IPython.display import display
url = 'https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystems.jsp?PointOfContactType=none&number=&WaterSystemStatus=A&name=&county=San%20Bernardino'
# last 3 rows arent systems
df = pd.read_html(url)[0][:-3]
print(len(df))
display(df.head(2))
display(df.tail(2))

361


,Water System No.,Water System Name,Type,Distribution System Class,Max Treatment Plant Class,Principal County Served,Primary Source Water Type
0,CA3601189,8986 DEEP CREEK ROAD,NC,NR,There are no treatment plants,SAN BERNARDINO,GW
1,CA3601171,ABRAHAM MAIDA WATER SYSTEM,NC,NR,There are no treatment plants,SAN BERNARDINO,GW


,Water System No.,Water System Name,Type,Distribution System Class,Max Treatment Plant Class,Principal County Served,Primary Source Water Type
359,CA3690019,YUCAIPA VALLEY WATER DIST. RECYCLED WATE,NP,NaN,There are no treatment plants,SAN BERNARDINO,NaN
360,CA3610055,YUCAIPA VALLEY WATER DISTRICT,C,D4,T3,SAN BERNARDINO,SW


In [6]:
systems = [
    "WEST END CONSOLIDATED WATER COMPANY",
    "WEST END MWC",
    "CUCAMONGA VALLEY WATER DISTRICT",
    "CUCAMONGA CWD-RECLAMATION",
    "SAN ANTONIO WATER COMPANY",
]
water_systems = df.pipe(lambda df: df.loc[df['Water System Name'].isin(systems)])
water_system_numbers = [i for i in water_systems['Water System No.']]
water_system_numbers

['CA3690016', 'CA3610018', 'CA3610085', 'CA3610086', 'CA3600345']

In [7]:
from pathlib import Path
import polars as pl

In [8]:
folder_path = Path(r"C:\Users\Denver\OneDrive - PROVOST & PRITCHARD CONSULTING GROUP\Desktop\SWRCB_WQ")
dfs = [
	pl.read_csv(
		file_path,
		# Default is , but the data is tab delimited
		sep="\t",
		# characters that are not valid utf8 will be ignored
		ignore_errors=True,
		# this tries to guess what the data is but we will assume it is all strings
		infer_schema_length=0
		# Assume all data is strings and remove leading and trailing whitespace
		).with_columns(
			pl.all().cast(pl.Utf8).str.strip() , strict=False
			# ).filter(pl.col('PS Code').str.strip().is_in(ps_codes))
			).filter(pl.col('Water System Number').str.strip().is_in(water_system_numbers))
		# do this for all .tab files in folder
		for file_path in folder_path.glob('*.tab')
		]

In [10]:
# combine all the dataframes into one
df = pl.concat(dfs)
print(df.shape)
df.head(2)

(53474, 29)


Regulating Agency,Water System Number,Water System Name,System Status,Water System Classification,Principal County Served,Population Served,Service Connections,PS Code,Sampling Point Name,Facility Type,Facility Status,Sample Type,Sample Date,Sample Time,Analysis Date,ELAP Cert#,Lab Name,Analyte Code,Analyte Name,Result,Counting Error,Units of Measure,Less Than Reporting Level,Reporting Level,DLR,MCL,Method,strict
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool
"""DISTRICT 13 - ...","""CA3610018""","""CUCAMONGA VALL...","""A""","""C""","""SAN BERNARDINO...","""200613""","""48283""","""CA3610018_001_...","""BEE TUNNEL""","""WL""","""A""","""RT""","""05-11-2012""","""08:45:00""","""05-15-2012""","""1088""","""CLINICAL LABOR...","""1021""","""HYDROXIDE AS C...",null,null,"""MG/L""","""Y""","""0.000000000""",null,null,null,false
"""DISTRICT 13 - ...","""CA3610018""","""CUCAMONGA VALL...","""A""","""C""","""SAN BERNARDINO...","""200613""","""48283""","""CA3610018_001_...","""BEE TUNNEL""","""WL""","""A""","""RT""","""05-11-2012""","""08:45:00""","""05-15-2012""","""1088""","""CLINICAL LABOR...","""1028""","""IRON""",null,null,"""UG/L""","""Y""","""100.000000000""","""100""","""300""",null,false


In [11]:
save_path = Path("data")
# save as parquet
df.write_parquet(save_path.joinpath("sdwis_wq.parquet"))